In [27]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

In [46]:
from pathlib import Path

roles = ["sys","human"]
levels = ["hard", "medium", "easy"]
question_format_template   = Path("question_format_template.txt").read_text().strip()

prompt = {}
for role in roles:
    prompt[role] = {}
    for level in levels:
        prompt_text = Path(role + "_template_"+level+".txt").read_text().strip()
        if (role == "human"):
            prompt_text += question_format_template
        prompt[role][level] = prompt_text


In [57]:
import json
def parse_question(chat_response):
    response = chat_response.replace("\n","")
    json_string = response.split("{")[1].split("}")[0]
    question = json.loads("{" + json_string + "}")
    return question

In [58]:
def create_question(context, level="hard"):
  completion = client.chat.completions.create(
    model="LM Studio Community/Meta-Llama-3-8B-Instruct-GGUF",
    messages=[
      {"role": "system", "content": prompt["sys"][level]},
      {"role": "user", "content": prompt["human"][level].replace('{context}',context)}
    ],
    temperature=0.1,
  )

  response = completion.choices[0].message
  print(response)
  return parse_question(response.content)


In [60]:


sample = context = "La Wikipedia en español es la edición en español o castellano de Wikipedia. Al igual que las versiones existentes de Wikipedia en otros idiomas, es una enciclopedia de contenido libre, publicada en Internet bajo las licencias libres CC BY-SA 4.0 y GFDL. En la actualidad cuenta con 1 949 592 artículos, y es escrita por usuarios voluntarios, es decir, que cualquiera puede editar un artículo, corregirlo o ampliarlo. Los servidores son administrados por la Fundación Wikimedia, una organización sin ánimo de lucro cuya financiación se basa fundamentalmente en donaciones."
levels = ["hard", "medium", "easy"] 

for level in levels:
    print("making a " + level + " question ..")
    question = create_question(sample, level)
    print(json.dumps(question,indent=2, ensure_ascii=False))


making a hard question ..
ChatCompletionMessage(content='{\n  "PREGUNTA": "¿Cómo se financia la Fundación Wikimedia?",\n  "OPCION1": "A través de publicidad en línea",\n  "OPCION2": "Vendiendo contenidos a empresas",\n  "OPCION3": "Recibiendo donaciones de particulares y organizaciones",\n  "OPCION4": "Fundada por una empresa tecnológica",\n  "EVIDENCIA": "La Fundación Wikimedia se financia fundamentalmente en donaciones, según el texto."\n}', role='assistant', function_call=None, tool_calls=None)
{
  "PREGUNTA": "¿Cómo se financia la Fundación Wikimedia?",
  "OPCION1": "A través de publicidad en línea",
  "OPCION2": "Vendiendo contenidos a empresas",
  "OPCION3": "Recibiendo donaciones de particulares y organizaciones",
  "OPCION4": "Fundada por una empresa tecnológica",
  "EVIDENCIA": "La Fundación Wikimedia se financia fundamentalmente en donaciones, según el texto."
}
making a medium question ..
ChatCompletionMessage(content='A continuación, te presento la pregunta y las opciones g